<a href="https://colab.research.google.com/github/LxYuan0420/nlp/blob/main/notebooks/Fine_tune_DistilGPT2_language_model_on_wikitext_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [ ]:
! pip install datasets transformers

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Then you need to install Git-LFS. Uncomment the following instructions:

In [3]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [4]:
import transformers

print(transformers.__version__)

4.28.1


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/language-modeling).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [7]:
from transformers.utils import send_example_telemetry

send_example_telemetry("language_modeling_notebook", framework="pytorch")

In [6]:
!nvidia-smi

Sun Apr 23 02:45:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Fine-tuning a language model

In this notebook, we'll see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model on a language modeling tasks. We will cover two types of language modeling tasks which are:

- Causal language modeling: the model has to predict the next token in the sentence (so the labels are the same as the inputs shifted to the right). To make sure the model does not cheat, it gets an attention mask that will prevent it to access the tokens after token i when trying to predict the token i+1 in the sentence.

![Widget inference representing the causal language modeling task](https://github.com/huggingface/notebooks/blob/master/examples/images/causal_language_modeling.png?raw=1)

We will see how to easily load and preprocess the dataset for each one of those tasks, and how to use the `Trainer` API to fine-tune a model on it.

A script version of this notebook you can directly run on a distributed environment or on TPU is available in our [examples folder](https://github.com/huggingface/transformers/tree/master/examples).

## Preparing the dataset

For each of those tasks, we will use the [Wikitext 2]() dataset as an example. You can load it very easily with the 🤗 Datasets library.

In [8]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

You can replace the dataset above with any dataset hosted on [the hub](https://huggingface.co/datasets) or use your own files. Just uncomment the following cell and replace the paths with values that will lead to your files:

In [ ]:
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

To access an actual element, you need to select a split first, then give an index:

In [10]:
datasets

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [11]:
datasets["train"][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

In [14]:
datasets["train"].features.items()

dict_items([('text', Value(dtype='string', id=None))])

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [15]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the datasets."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [17]:
show_random_elements(datasets["train"])

,text
0,"In a February 4 , 2008 , article , Scientology spokeswoman Karin Pouw told the Los Angeles Times that Church of Scientology 's websites "" have been and are online . "" Danny McPherson , chief research officer at Arbor Networks , claimed 500 denial @-@ of @-@ service attacks had been observed on the Scientology site in the week prior to February 4 , some of which were strong enough to bring the website down . Calling Anonymous a "" motley crew of internet troublemakers "" , Wired blogger Ryan Singel said that , while attempting to bypass the Prolexic servers protecting the Church of Scientology website , users of a misconfigured DDoS tool inadvertently and briefly had targeted the Etty Hillesum Lyceum , a Dutch secondary school in Deventer . Another hacking group associated with the project , calling themselves the "" g00ns "" , mistakenly targeted a 59 @-@ year @-@ old man from Stockton , California . They posted his home telephone number , address and his wife 's Social Security number online for other people to target . They believed that he was behind counter @-@ attacks against Project Chanology @-@ related websites by the Regime , a counter @-@ hack group who crashed one of the Project Chanology planning websites . The group allegedly attempted to gain personal information on people involved in Project Chanology to turn that information over to the Church of Scientology . After discovering they had wrongly targeted the couple , one of the members of the g00ns group called and apologized . \n"
1,"Rachel is considered to be Aniston 's breakout role , credited with making her the show 's most famous cast member and for spawning her successful film career . Praised for her performance as Rachel , Aniston won both an Emmy Award for Outstanding Lead Actress in a Comedy Series and a Golden Globe Award for Best Performance by an Actress In A Television Series – Comedy Or Musical . \n"
2,"Is heap 'd upon her , maiden most unmeek , — \n"
3,"Florida Atlantic has changed dramatically since its opening in 1964 . As of 2013 , there are more than 30 @,@ 000 students attending classes on seven campuses spread across 120 miles ( 193 km ) . The university consists of ten colleges and employs more than 3 @,@ 200 faculty and staff . The university 's endowment decreased from $ 182 million in June 2008 to $ 142 million in January 2009 due to a worsening economy . However , with the national economic recovery the endowment has increased to $ 179 million by the end of 2012 . \n"
4,
5,"The skeleton of the kakapo differs from other parrots in several features associated with flightlessness . Firstly , it has the smallest relative wing size of any parrot . Its wing feathers are shorter , more rounded , and less asymmetrical and have fewer distal barbules to lock the feathers together . The sternum is small and has a low , vestigial keel and a shortened spina externa . As in other flightless birds and some other flighted parrots , the furcula is not fused but consists of a pair of clavicles lying in contact with each coracoid . As in other flightless birds , the angle between the coracoid and sternum is enlarged . The kakapo has a larger pelvis than other parrots . The proximal bones of the leg and arm are disproportionately long and the distal elements are disproportionately short . \n"
6,"The corn crake 's breeding habitat is grassland , particularly hayfields , and it uses similar environments on the wintering grounds . This secretive species builds a nest of grass leaves in a hollow in the ground and lays 6 – 14 cream @-@ coloured eggs which are covered with rufous blotches . These hatch in 19 – 20 days , and the black precocial chicks fledge after about five weeks . This crake is in steep decline across much of its former breeding range because modern farming practices often destroy nests before breeding is completed . The corn crake is omnivorous but mainly feeds on invertebrates , the occasional small frog or mammal , and plant mat

As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines.

## Causal Language modeling

For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text that may look like:
```
part of text 1
```
or 
```
end of text 1 [BOS_TOKEN] beginning of text 2
```
depending on whether they span over several of the original texts in the dataset or not. The labels will be the same as the inputs, shifted to the left.

We will use the [`distilgpt2`](https://huggingface.co/distilgpt2) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=causal-lm) instead:

In [18]:
model_checkpoint = "distilgpt2"

To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

In [19]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

In [22]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

In [23]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

If we now look at an element of our datasets, we will see the text have been replaced by the `input_ids` the model will need:

In [24]:
tokenized_datasets["train"][1]

{'input_ids': [796, 569, 18354, 7496, 17740, 6711, 796, 220, 198],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [26]:
tokenized_datasets

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 3760
    })
})

Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain `block_size`. To do this, we will use the `map` method again, with the option `batched=True`. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.

First, we grab the maximum length our model was pretrained with. This might be a big too big to fit in your GPU RAM, so here we take a bit less at just 128.

In [27]:
tokenizer.model_max_length

1024

In [28]:
# block_size = tokenizer.model_max_length
block_size = 128

Then we write the preprocessing function that will group our texts:

In [35]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

First note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.

Also note that by default, the `map` method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized texts a multiple of `block_size` every 1,000 examples. You can adjust this behavior by passing a higher batch size (which will also be processed slower). You can also speed-up the preprocessing by using multiprocessing:

In [36]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

And we can check our datasets have changed: now the samples contain chunks of `block_size` contiguous tokens, potentially spanning over several of our original texts.

In [39]:
lm_datasets

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2209
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 18666
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1931
    })
})

In [42]:
tokenizer.decode(lm_datasets["train"][0]["input_ids"])

' = Valkyria Chronicles III = \n Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3, lit. Valkyria of the Battlefield 3 ), commonly referred to as Valkyria Chronicles III outside Japan, is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable. Released in January 2011 in Japan, it is the third game in the Valkyria series. Employing the same fusion of tactical and real @-@ time gameplay as its predecessors, the story runs parallel to the first'

In [41]:
tokenizer.decode(lm_datasets["train"][0]["labels"])

' = Valkyria Chronicles III = \n Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3, lit. Valkyria of the Battlefield 3 ), commonly referred to as Valkyria Chronicles III outside Japan, is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable. Released in January 2011 in Japan, it is the third game in the Valkyria series. Employing the same fusion of tactical and real @-@ time gameplay as its predecessors, the story runs parallel to the first'

In [45]:
tokenizer("Hello World")

{'input_ids': [15496, 2159], 'attention_mask': [1, 1]}

In [48]:
tokenizer.decode(tokenizer("Hello World")["input_ids"])

'Hello World'

In [47]:
tokenizer.decode(tokenizer.encode("Hello World"))

'Hello World'

Now that the data has been cleaned, we're ready to instantiate our `Trainer`. We will a model:

In [49]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

And some `TrainingArguments`:

In [50]:
from transformers import Trainer, TrainingArguments

In [54]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    num_train_epochs=6,
    push_to_hub=True,
)

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/gpt-finetuned-wikitext2"` or `"huggingface/gpt-finetuned-wikitext2"`).

We pass along all of those to the `Trainer` class:

In [55]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

/content/distilgpt2-finetuned-wikitext2 is already a clone of https://huggingface.co/lxyuan/distilgpt2-finetuned-wikitext2. Make sure you pull the latest changes with `repo.git_pull()`.


And we can train our model:

In [56]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.793000,3.678605


Epoch,Training Loss,Validation Loss
1,3.793000,3.678605
2,3.703900,3.656468
3,3.652300,3.645229
4,3.615700,3.641571
5,3.590700,3.637758
6,3.563500,3.637481


TrainOutput(global_step=3504, training_loss=3.64134884615467, metrics={'train_runtime': 2855.1041, 'train_samples_per_second': 39.227, 'train_steps_per_second': 1.227, 'total_flos': 3658023859912704.0, 'train_loss': 3.64134884615467, 'epoch': 6.0})

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [57]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 38.00


You can now upload the result of the training to the Hub, just execute this instruction:

In [58]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/318M [00:00<?, ?B/s]

Upload file runs/Apr23_03-16-24_097e8565bc69/events.out.tfevents.1682222666.097e8565bc69.200.4:   0%|         …

Upload file runs/Apr23_03-16-24_097e8565bc69/events.out.tfevents.1682219791.097e8565bc69.200.2:   0%|         …

To https://huggingface.co/lxyuan/distilgpt2-finetuned-wikitext2
   2ca3176..e19d1ad  main -> main

   2ca3176..e19d1ad  main -> main

To https://huggingface.co/lxyuan/distilgpt2-finetuned-wikitext2
   e19d1ad..0b1109b  main -> main

   e19d1ad..0b1109b  main -> main



'https://huggingface.co/lxyuan/distilgpt2-finetuned-wikitext2/commit/e19d1ad7bc9deb62031f608b9f3db5c0b92698b4'

In [59]:
tokenizer.push_to_hub("distilgpt2-finetuned-wikitext2")

CommitInfo(commit_url='https://huggingface.co/lxyuan/distilgpt2-finetuned-wikitext2/commit/aa02c333e13819a0e97a149fc491586dbb245b4c', commit_message='Upload tokenizer', commit_description='', oid='aa02c333e13819a0e97a149fc491586dbb245b4c', pr_url=None, pr_revision=None, pr_num=None)

#### Download our model from HF and try it using pipeline

In [81]:
from transformers import pipeline

generator = pipeline(model="lxyuan/distilgpt2-finetuned-wikitext2")

In [85]:
generator("Lion King is", pad_token_id=generator.tokenizer.eos_token_id, max_new_tokens=50, num_return_sequences=2)

[{'generated_text': 'Lion King is the only king of the two lands currently controlled by the emperor. King Henry II has since ruled through the Middle Ages, most recently after his death. Queen Edward I of England was assassinated in 1765. Edward II died in 1666. \n'},
 {'generated_text': 'Lion King is a playable version of the main story arc for The New Adventures of Magic. An updated version features two characters on a different character : the Dark Knight ( Shadow Knight ) and the Knights of the Temple of the Shadow ( Z @-@ Knight ).'}]